![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Vector Search with RedisVL

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/01_redisvl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Prepare data

In this examples we will load a list of movies with the following attributes: `title`, `rating`, `description`, and `genre`.

We will embed the movie description so that user's can search for movies that best match the kind of movie that they're looking for.

**If you are running this notebook locally**, FYI you may not need to perform this step at all.

In [25]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

Cloning into 'temp_repo'...
remote: Enumerating objects: 797, done.
remote: Counting objects: 100% (337/337), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 797 (delta 248), reused 147 (delta 131), pack-reused 460 (from 3)
Receiving objects: 100% (797/797), 58.59 MiB | 37.13 MiB/s, done.
Resolving deltas: 100% (357/357), done.
mv: cannot move 'temp_repo/python-recipes/vector-search/resources' to './resources': Directory not empty


## Packages

In [26]:
%pip install -q "redisvl>=0.6.0" sentence-transformers pandas nltk

## Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [27]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


gpg: cannot open '/dev/tty': No such device or address
curl: (23) Failed writing body


#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [28]:
import os
import warnings

warnings.filterwarnings('ignore')

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client

In [29]:
from redis import Redis

client = Redis.from_url(REDIS_URL)
client.ping()

True

In [30]:
#client.flushall()

### Load Movies Dataset

In [31]:
data = [
  {
    "id": 1,
    "title": "צימר בגליל באילת",
    "description": "חוויה קולינרית/אירוח צימר בעיר אילת. המקום הכי מדובר באזור.",
    "tags": "צימר בגליל,צימר,אירוח כפרי,אילת,eilat",
    "rating": 10
  },
  {
    "id": 2,
    "title": "ראמן באילת",
    "description": "חוויה קולינרית/אירוח יפני בעיר אילת. המקום הכי מדובר באזור.",
    "tags": "ראמן,יפני,street food,אילת,eilat",
    "rating": 8
  },
  {
    "id": 3,
    "title": "טאקו בחיפה",
    "description": "חוויה קולינרית/אירוח מקסיקני בעיר חיפה. המקום הכי מדובר באזור.",
    "tags": "טאקו,מקסיקני,mexican,חיפה,haifa",
    "rating": 9
  },
  {
    "id": 4,
    "title": "סטייק בהרצליה",
    "description": "חוויה קולינרית/אירוח בשרי בעיר הרצליה. המקום הכי מדובר באזור.",
    "tags": "סטייק,בשרי,steakhouse,הרצליה,herzliya",
    "rating": 8
  },
  {
    "id": 5,
    "title": "שווארמה בהרצליה",
    "description": "חוויה קולינרית/אירוח מזרחי בעיר הרצליה. המקום הכי מדובר באזור.",
    "tags": "שווארמה,מזרחי,middle eastern,הרצליה,herzliya",
    "rating": 6
  },
  {
    "id": 6,
    "title": "מלון יוקרה בנתניה",
    "description": "חוויה קולינרית/אירוח מלון בעיר נתניה. המקום הכי מדובר באזור.",
    "tags": "מלון יוקרה,מלון,luxury,נתניה,netanya",
    "rating": 6
  },
  {
    "id": 7,
    "title": "פיצה ברמת גן",
    "description": "חוויה קולינרית/אירוח איטלקי בעיר רמת גן. המקום הכי מדובר באזור.",
    "tags": "פיצה,איטלקי,pizza,רמת גן,ramat gan",
    "rating": 8
  },
  {
    "id": 8,
    "title": "מלון בוטיק בהרצליה",
    "description": "חוויה קולינרית/אירוח מלון בעיר הרצליה. המקום הכי מדובר באזור.",
    "tags": "מלון בוטיק,מלון,boutique hotel,הרצליה,herzliya",
    "rating": 10
  },
  {
    "id": 9,
    "title": "שווארמה ברמת גן",
    "description": "חוויה קולינרית/אירוח מזרחי בעיר רמת גן. המקום הכי מדובר באזור.",
    "tags": "שווארמה,מזרחי,middle eastern,רמת גן,ramat gan",
    "rating": 7
  },
  {
    "id": 10,
    "title": "ראמן באילת",
    "description": "חוויה קולינרית/אירוח יפני בעיר אילת. המקום הכי מדובר באזור.",
    "tags": "ראמן,יפני,street food,אילת,eilat",
    "rating": 6
  },
  {
    "id": 11,
    "title": "המבורגר בתל אביב",
    "description": "חוויה קולינרית/אירוח אמריקאי בעיר תל אביב. המקום הכי מדובר באזור.",
    "tags": "המבורגר,אמריקאי,fast food,תל אביב,tel aviv",
    "rating": 9
  },
  {
    "id": 12,
    "title": "המבורגר בירושלים",
    "description": "חוויה קולינרית/אירוח אמריקאי בעיר ירושלים. המקום הכי מדובר באזור.",
    "tags": "המבורגר,אמריקאי,fast food,ירושלים,jerusalem",
    "rating": 10
  },
  {
    "id": 13,
    "title": "פלאפל בתל אביב",
    "description": "חוויה קולינרית/אירוח ישראלי בעיר תל אביב. המקום הכי מדובר באזור.",
    "tags": "פלאפל,ישראלי,vegetarian,תל אביב,tel aviv",
    "rating": 10
  },
  {
    "id": 14,
    "title": "שווארמה באילת",
    "description": "חוויה קולינרית/אירוח מזרחי בעיר אילת. המקום הכי מדובר באזור.",
    "tags": "שווארמה,מזרחי,middle eastern,אילת,eilat",
    "rating": 7
  },
  {
    "id": 15,
    "title": "אכסניה בירושלים",
    "description": "חוויה קולינרית/אירוח הוסטל בעיר ירושלים. המקום הכי מדובר באזור.",
    "tags": "אכסניה,הוסטל,hostel,ירושלים,jerusalem",
    "rating": 9
  },
  {
    "id": 16,
    "title": "פיצה בהרצליה",
    "description": "חוויה קולינרית/אירוח איטלקי בעיר הרצליה. המקום הכי מדובר באזור.",
    "tags": "פיצה,איטלקי,pizza,הרצליה,herzliya",
    "rating": 9
  },
  {
    "id": 17,
    "title": "סלט בתל אביב",
    "description": "חוויה קולינרית/אירוח בריא בעיר תל אביב. המקום הכי מדובר באזור.",
    "tags": "סלט,בריא,healthy,תל אביב,tel aviv",
    "rating": 6
  },
  {
    "id": 18,
    "title": "ראמן בבאר שבע",
    "description": "חוויה קולינרית/אירוח יפני בעיר באר שבע. המקום הכי מדובר באזור.",
    "tags": "ראמן,יפני,street food,באר שבע,beer sheva",
    "rating": 10
  },
  {
    "id": 19,
    "title": "סושי בבאר שבע",
    "description": "חוויה קולינרית/אירוח יפני בעיר באר שבע. המקום הכי מדובר באזור.",
    "tags": "סושי,יפני,asian,באר שבע,beer sheva",
    "rating": 6
  },
  {
    "id": 20,
    "title": "סושי בחיפה",
    "description": "חוויה קולינרית/אירוח יפני בעיר חיפה. המקום הכי מדובר באזור.",
    "tags": "סושי,יפני,asian,חיפה,haifa",
    "rating": 9
  },
  {
    "id": 21,
    "title": "צימר בגליל בירושלים",
    "description": "חוויה קולינרית/אירוח צימר בעיר ירושלים. המקום הכי מדובר באזור.",
    "tags": "צימר בגליל,צימר,אירוח כפרי,ירושלים,jerusalem",
    "rating": 9
  },
  {
    "id": 22,
    "title": "מלון יוקרה בתל אביב",
    "description": "חוויה קולינרית/אירוח מלון בעיר תל אביב. המקום הכי מדובר באזור.",
    "tags": "מלון יוקרה,מלון,luxury,תל אביב,tel aviv",
    "rating": 9
  },
  {
    "id": 23,
    "title": "מלון יוקרה בתל אביב",
    "description": "חוויה קולינרית/אירוח מלון בעיר תל אביב. המקום הכי מדובר באזור.",
    "tags": "מלון יוקרה,מלון,luxury,תל אביב,tel aviv",
    "rating": 6
  },
  {
    "id": 24,
    "title": "סלט בבאר שבע",
    "description": "חוויה קולינרית/אירוח בריא בעיר באר שבע. המקום הכי מדובר באזור.",
    "tags": "סלט,בריא,healthy,באר שבע,beer sheva",
    "rating": 6
  },
  {
    "id": 25,
    "title": "המבורגר בהרצליה",
    "description": "חוויה קולינרית/אירוח אמריקאי בעיר הרצליה. המקום הכי מדובר באזור.",
    "tags": "המבורגר,אמריקאי,fast food,הרצליה,herzliya",
    "rating": 7
  },
  {
    "id": 26,
    "title": "צימר בגליל ברמת גן",
    "description": "חוויה קולינרית/אירוח צימר בעיר רמת גן. המקום הכי מדובר באזור.",
    "tags": "צימר בגליל,צימר,אירוח כפרי,רמת גן,ramat gan",
    "rating": 6
  },
  {
    "id": 27,
    "title": "פלאפל בחיפה",
    "description": "חוויה קולינרית/אירוח ישראלי בעיר חיפה. המקום הכי מדובר באזור.",
    "tags": "פלאפל,ישראלי,vegetarian,חיפה,haifa",
    "rating": 8
  },
  {
    "id": 28,
    "title": "פיצה בירושלים",
    "description": "חוויה קולינרית/אירוח איטלקי בעיר ירושלים. המקום הכי מדובר באזור.",
    "tags": "פיצה,איטלקי,pizza,ירושלים,jerusalem",
    "rating": 8
  },
  {
    "id": 29,
    "title": "פלאפל בירושלים",
    "description": "חוויה קולינרית/אירוח ישראלי בעיר ירושלים. המקום הכי מדובר באזור.",
    "tags": "פלאפל,ישראלי,vegetarian,ירושלים,jerusalem",
    "rating": 6
  },
  {
    "id": 30,
    "title": "אכסניה בתל אביב",
    "description": "חוויה קולינרית/אירוח הוסטל בעיר תל אביב. המקום הכי מדובר באזור.",
    "tags": "אכסניה,הוסטל,hostel,תל אביב,tel aviv",
    "rating": 9
  },
  {
    "id": 31,
    "title": "פלאפל בהרצליה",
    "description": "חוויה קולינרית/אירוח ישראלי בעיר הרצליה. המקום הכי מדובר באזור.",
    "tags": "פלאפל,ישראלי,vegetarian,הרצליה,herzliya",
    "rating": 9
  },
  {
    "id": 32,
    "title": "מלון בוטיק בנתניה",
    "description": "חוויה קולינרית/אירוח מלון בעיר נתניה. המקום הכי מדובר באזור.",
    "tags": "מלון בוטיק,מלון,boutique hotel,נתניה,netanya",
    "rating": 8
  },
  {
    "id": 33,
    "title": "טאקו בירושלים",
    "description": "חוויה קולינרית/אירוח מקסיקני בעיר ירושלים. המקום הכי מדובר באזור.",
    "tags": "טאקו,מקסיקני,mexican,ירושלים,jerusalem",
    "rating": 8
  },
  {
    "id": 34,
    "title": "פיצה בהרצליה",
    "description": "חוויה קולינרית/אירוח איטלקי בעיר הרצליה. המקום הכי מדובר באזור.",
    "tags": "פיצה,איטלקי,pizza,הרצליה,herzliya",
    "rating": 6
  },
  {
    "id": 35,
    "title": "אכסניה בירושלים",
    "description": "חוויה קולינרית/אירוח הוסטל בעיר ירושלים. המקום הכי מדובר באזור.",
    "tags": "אכסניה,הוסטל,hostel,ירושלים,jerusalem",
    "rating": 7
  },
  {
    "id": 36,
    "title": "סלט באילת",
    "description": "חוויה קולינרית/אירוח בריא בעיר אילת. המקום הכי מדובר באזור.",
    "tags": "סלט,בריא,healthy,אילת,eilat",
    "rating": 9
  },
  {
    "id": 37,
    "title": "צימר בגליל בירושלים",
    "description": "חוויה קולינרית/אירוח צימר בעיר ירושלים. המקום הכי מדובר באזור.",
    "tags": "צימר בגליל,צימר,אירוח כפרי,ירושלים,jerusalem",
    "rating": 8
  },
  {
    "id": 38,
    "title": "טאקו בתל אביב",
    "description": "חוויה קולינרית/אירוח מקסיקני בעיר תל אביב. המקום הכי מדובר באזור.",
    "tags": "טאקו,מקסיקני,mexican,תל אביב,tel aviv",
    "rating": 8
  },
  {
    "id": 39,
    "title": "שווארמה בירושלים",
    "description": "חוויה קולינרית/אירוח מזרחי בעיר ירושלים. המקום הכי מדובר באזור.",
    "tags": "שווארמה,מזרחי,middle eastern,ירושלים,jerusalem",
    "rating": 7
  },
  {
    "id": 40,
    "title": "מלון יוקרה באילת",
    "description": "חוויה קולינרית/אירוח מלון בעיר אילת. המקום הכי מדובר באזור.",
    "tags": "מלון יוקרה,מלון,luxury,אילת,eilat",
    "rating": 8
  }
]


df = pd.DataFrame(data)
df


,id,title,description,tags,rating
0,1,סושי במסעדת יאמאטה,הצצנו מאחורי הקלעים של יאמאטה בתל אביב – מנת ס...,"יאמאטה,סושי,יפני,תל אביב,מסעדה",9
1,2,המבורגר אמריקאי בשינקין,המבורגר עסיסי בסגנון דיינר עם גבינה נמסה וצ'יפ...,"המבורגר,אמריקאי,fast food,שינקין,תל אביב",8
2,3,ראמן בדאון טאון חיפה,מסעדת רחוב שמגישה ראמן חם עם טעמים עשירים. חוו...,"ראמן,יפני,חיפה,אסיאתי,street food",7
3,4,סושי ביפו העתיקה,סיבוב קולינרי בשוק הפשפשים – טעמנו רולים פריכי...,"סושי,יפו,תל אביב,מסעדה,רול",8
4,5,Tuna Bowl at Toki,Healthy tuna poke bowl with avocado and mango ...,"poke,tuna,sarona,tel aviv,healthy",9


In [32]:
import pandas as pd
import numpy as np
import json

# df = pd.read_json("resources/movies.json")
print("Loaded", len(df), "movie entries")

df.head()

Loaded 5 movie entries


,id,title,description,tags,rating
0,1,סושי במסעדת יאמאטה,הצצנו מאחורי הקלעים של יאמאטה בתל אביב – מנת ס...,"יאמאטה,סושי,יפני,תל אביב,מסעדה",9
1,2,המבורגר אמריקאי בשינקין,המבורגר עסיסי בסגנון דיינר עם גבינה נמסה וצ'יפ...,"המבורגר,אמריקאי,fast food,שינקין,תל אביב",8
2,3,ראמן בדאון טאון חיפה,מסעדת רחוב שמגישה ראמן חם עם טעמים עשירים. חוו...,"ראמן,יפני,חיפה,אסיאתי,street food",7
3,4,סושי ביפו העתיקה,סיבוב קולינרי בשוק הפשפשים – טעמנו רולים פריכי...,"סושי,יפו,תל אביב,מסעדה,רול",8
4,5,Tuna Bowl at Toki,Healthy tuna poke bowl with avocado and mango ...,"poke,tuna,sarona,tel aviv,healthy",9


In [33]:
from redisvl.utils.vectorize import HFTextVectorizer
from redisvl.extensions.cache.embeddings import EmbeddingsCache

os.environ["TOKENIZERS_PARALLELISM"] = "false"


hf = HFTextVectorizer(
    model="sentence-transformers/all-MiniLM-L6-v2",
    cache=EmbeddingsCache(
        name="embedcache",
        ttl=600,
        redis_client=client,
    )
)

In [34]:
# Generate vectors
df["vector"] = hf.embed_many(df["description"].tolist(), as_buffer=True)

df.head()

,id,title,description,tags,rating,vector
0,1,סושי במסעדת יאמאטה,הצצנו מאחורי הקלעים של יאמאטה בתל אביב – מנת ס...,"יאמאטה,סושי,יפני,תל אביב,מסעדה",9,b'\x0e8j=O\x07\x06>\xa1\xb9 <\xd2\xae\x83<}\xf...
1,2,המבורגר אמריקאי בשינקין,המבורגר עסיסי בסגנון דיינר עם גבינה נמסה וצ'יפ...,"המבורגר,אמריקאי,fast food,שינקין,תל אביב",8,b'\x13\xd9\x00\xbd\xbe\xf7\xdf=f\xd7\xe8\xbb9\...
2,3,ראמן בדאון טאון חיפה,מסעדת רחוב שמגישה ראמן חם עם טעמים עשירים. חוו...,"ראמן,יפני,חיפה,אסיאתי,street food",7,"b'\xe3\x86\x88:\xcbq\xb3=\xf2\xda,\xbc\xe3\xed..."
3,4,סושי ביפו העתיקה,סיבוב קולינרי בשוק הפשפשים – טעמנו רולים פריכי...,"סושי,יפו,תל אביב,מסעדה,רול",8,b'P\x9f\x9d\xbcm\tL=\xc4\xd5\x82\xbb\xe8\xe10=...
4,5,Tuna Bowl at Toki,Healthy tuna poke bowl with avocado and mango ...,"poke,tuna,sarona,tel aviv,healthy",9,b't\xe8\x8c\xbc6\xdb\x9d<\x811\xf5\xbc=\x18_=f...


## Define Redis index schema

In [35]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex


index_name = "movies"

schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
    "prefix": index_name,
    "storage_type": "hash"
  },
  "fields": [
    {
        "name": "title",
        "type": "text",
    },
    {
        "name": "description",
        "type": "text",
    },
    {
        "name": "genre",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "rating",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "flat",
            "datatype": "float32"
        }
    }
  ]
})


index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

In [36]:
!rvl index info -i movies -u {REDIS_URL}



Index Information:
╭───────────────┬───────────────┬───────────────┬───────────────┬───────────────┬╮
│ Index Name    │ Storage Type  │ Prefixes      │ Index Options │ Indexing      │
├───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼┤
| movies        | HASH          | ['movies']    | []            | 0             |
╰───────────────┴───────────────┴───────────────┴───────────────┴───────────────┴╯
Index Fields:
╭─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┬╮
│ Name            │ Attribute       │ Type            │ Field Option    │ Option Value    │ Field Option    │ Option Value    │ Field Option    │ Option Value    │ Field Option    │ Option Value    │
├─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼─────────────────┼──

## Populate index

In [37]:
index.load(df.to_dict(orient="records"))

['movies:01K17ZM1JJ68XA9VBY5WG994FF',
 'movies:01K17ZM1JJCAZDZHW3S3QWD6C8',
 'movies:01K17ZM1JJJWB3210387A0P4SN',
 'movies:01K17ZM1JJSA1T7HK7VFS4H568',
 'movies:01K17ZM1JJX0782G2T1P22QRC5']

## Search techniques

### Standard vector search

In [56]:
from redisvl.query import VectorQuery
from redisvl.query.filter import Tag  # if you want tag filtering later

user_query = "המבורגר בתל אביב"
embedded_user_query = hf.embed(user_query)

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "description", "tags", "rating"],
    return_score=True,
)

result = index.query(vec_query)
pd.DataFrame(result)


,id,vector_distance,title,description,tags,rating
0,movies:01K17ZM1JJ68XA9VBY5WG994FF,0.468630731106,סושי במסעדת יאמאטה,הצצנו מאחורי הקלעים של יאמאטה בתל אביב – מנת ס...,"יאמאטה,סושי,יפני,תל אביב,מסעדה",9
1,movies:01K17ZM1JJSA1T7HK7VFS4H568,0.645182609558,סושי ביפו העתיקה,סיבוב קולינרי בשוק הפשפשים – טעמנו רולים פריכי...,"סושי,יפו,תל אביב,מסעדה,רול",8
2,movies:01K17ZM1JJJWB3210387A0P4SN,0.647178053856,ראמן בדאון טאון חיפה,מסעדת רחוב שמגישה ראמן חם עם טעמים עשירים. חוו...,"ראמן,יפני,חיפה,אסיאתי,street food",7


### Vector search with filters

Redis allows you to combine filter searches on fields within the index object allowing us to create more specific searches.

Search for top 3 movies specifically in the action genre:


In [39]:
from redisvl.query.filter import Tag

tag_filter = Tag("genre") == "action"

vec_query.set_filter(tag_filter)

result=index.query(vec_query)
pd.DataFrame(result)

""


Search for top 3 movies specifically in the action genre with ratings at or above a 7:


In [40]:
from redisvl.query.filter import Num

# build combined filter expressions
tag_filter = Tag("genre") == "action"
num_filter = Num("rating") >= 7
combined_filter = tag_filter & num_filter

# build vector query
vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre"],
    return_score=True,
    filter_expression=combined_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

""


Search with full text search for movies that directly mention "criminal mastermind" in the description:


In [41]:
from redisvl.query.filter import Text

text_filter = Text("description") % "criminal mastermind"

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True,
    filter_expression=text_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

""


Vector search with wildcard text match:


In [42]:
text_filter = Text("description") % "crim*"

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True,
    filter_expression=text_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

""


Vector search with fuzzy match filter

> Note: fuzzy match is based on Levenshtein distance. Therefore, "hero" might return result for "her" as an example.

See docs for more info https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/query_syntax/


In [43]:

text_filter = Text("description") % "%hero%"

vec_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    num_results=3,
    return_fields=["title", "rating", "genre", "description"],
    return_score=True,
    filter_expression=text_filter
)

result = index.query(vec_query)
pd.DataFrame(result)

""


### Range queries

Range queries allow you to set a pre defined distance "threshold" for which we want to return documents. This is helpful when you only want documents with a certain "radius" from the search query.

In [44]:
from redisvl.query import RangeQuery

user_query = "Family friendly fantasy movies"

embedded_user_query = hf.embed(user_query)

range_query = RangeQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    return_fields=["title", "rating", "genre"],
    return_score=True,
    distance_threshold=0.8  # find all items with a semantic distance of less than 0.8
)

result = index.query(range_query)
pd.DataFrame(result)


""


Like the queries above, we can also chain additional filters and conditional operators with range queries. The following adds an `and` condition that returns vector search within the defined range and with a rating at or above 8.

In [45]:
range_query = RangeQuery(
    vector=embedded_user_query,
    vector_field_name="vector",
    return_fields=["title", "rating", "genre"],
    distance_threshold=0.8
)

numeric_filter = Num("rating") >= 8

range_query.set_filter(numeric_filter)

# in this case we want to do a simple filter search or the vector so we execute as a joint filter directly
result = index.query(range_query)
pd.DataFrame(result)


""


### Full text search

In [46]:
from redisvl.query import TextQuery

user_query = "High tech, action packed, superheros fight scenes"

text_query = TextQuery(
    text=user_query,
    text_field_name="description",
    text_scorer="BM25STD",
    num_results=20,
    return_fields=["title", "description"],
)

result = index.query(text_query)[:4]
pd.DataFrame(result)[["title", "score"]]

KeyError: "None of [Index(['title', 'score'], dtype='object')] are in the [columns]"

### Hybrid search

In [ ]:
from redisvl.query import HybridQuery

hybrid_query = HybridQuery(
    text=user_query,
    text_field_name="description",
    text_scorer="BM25",
    vector=embedded_user_query,
    vector_field_name="vector",
    alpha=0.7,
    num_results=20,
    return_fields=["title", "description"],
)

result = index.query(hybrid_query)[:4]
pd.DataFrame(result)[["title", "vector_similarity", "text_score", "hybrid_score"]]

### Next steps

For more query examples with redisvl: [see here](https://github.com/redis/redis-vl-python/blob/main/docs/user_guide/02_hybrid_queries.ipynb)

In [ ]:
# clean up!
index.delete()